In [1]:
import torch
import pandas as pd
from PIL import Image
from lavis.models import load_model_and_preprocess
from glob import glob
from tqdm import tqdm

In [3]:
# setup device to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Load BLIP large captioning model finetuned on COCO

In [4]:
# we associate a model with its preprocessors to make it easier for inference.
model, vis_processors, _ = load_model_and_preprocess(
    name="blip_caption", model_type="large_coco", is_eval=True, device=device
)
# uncomment to use base model
# model, vis_processors, _ = load_model_and_preprocess(
#     name="blip_caption", model_type="base_coco", is_eval=True, device=device
# )
vis_processors.keys()

dict_keys(['train', 'eval'])

In [7]:
images = sorted(glob('../img_captioning_CNNRNN/images/train/*.jpg'))
captions = []
i=0
for image in tqdm(images):
    i+=1
    if i % 100 == 0:
        df = pd.DataFrame({'image': images[:i], 'caption': captions})
        df.to_csv(f'../captions_{i}.csv', index=False)
    raw_image = Image.open(image).convert('RGB')
    image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
    caption = model.generate({"image": image})
    captions.append(caption)
    
df = pd.DataFrame({'image': images, 'caption': captions})
df.to_csv('../captions.csv', index=False)

['there is a large body of water with a giant statue',
 'the statue is spewing water from a fountain on the river',
 'there are many fountains that come into the river']